# numerical range of density matrix

Comment: the numerical range in this page is actually "joint algebraic numerical range" to be precise, but we will just call it numerical range for simplicity. 

Density matrix is postive semi-definite, Hermitian, trace one matrix.

$$ \{ \rho\in\mathbb{C}^{d\times d}: \rho\succeq 0,\mathrm{Tr}[\rho]=1 \} $$

a positive semi-definite matrix is always Hermitian, so we don't need to specify it.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numqi


## one qubit case

For one qubit case, density matrix is a $2\times 2$ matrix, and it can be written as

$$ \rho=\rho_{0}+\frac{1}{2}\vec{n}\cdot\vec{\sigma} $$

where $\vec{n}\in \mathbb{R}^3$ is a three dimensional vector. To make sure the expression above PSD, the vector is required to be inside the Bloch sphere, i.e., $\lVert\vec{n}\rVert _2\leq 1$ [wiki-link](https://en.wikipedia.org/wiki/Bloch_sphere). And the numerical range of two operators is simply the projection of the Bloch sphere onto the plane spanned by the two operators. For this Bloch sphere, the projection is a circle if these two operators are perpendicular to each other.

In [ ]:
np_rng = np.random.default_rng()

# generate two random, orthogonal unit vectors
tmp0 = np_rng.uniform(-1, 1, size=3)
vec0 = tmp0 / np.linalg.norm(tmp0)
op0 = vec0[0] * numqi.gate.X + vec0[1] * numqi.gate.Y + vec0[2] * numqi.gate.Z

tmp1 = np_rng.uniform(-1, 1, size=3)
tmp1 = tmp1 - tmp1.dot(vec0) * vec0 # orthogonalize
vec1 = tmp1 / np.linalg.norm(tmp1)
op1 = vec1[0] * numqi.gate.X + vec1[1] * numqi.gate.Y + vec1[2] * numqi.gate.Z

fig,ax = plt.subplots()
theta_list = np.linspace(0, 2*np.pi, 200)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)
beta_list = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
ax.plot(beta_list*np.cos(theta_list), beta_list*np.sin(theta_list))
ax.set_aspect('equal')
ax.set_xlabel('$O_0$')
ax.set_ylabel('$O_1$')
_ = ax.set_title('two random orthogonal operators')


task: think about what happened if these two operators are not perpendicular to each other.

## two qubits case

For two qubits case, the shape of numerical range is much more complicated. Let's start with two commutative operators,

$$ O_0=X\otimes X,O_1=Z\otimes Z $$

one can verify that $[O_0,O_1]=0$.

In [ ]:
op0 = np.kron(numqi.gate.X, numqi.gate.X)
op1 = np.kron(numqi.gate.Z, numqi.gate.Z)
# op1 = np.kron(numqi.gate.Z, numqi.gate.I) + np.kron(numqi.gate.I, numqi.gate.Z)

theta_list = np.linspace(0, 2*np.pi, 400)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

beta_dm = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
nr_dm = beta_dm.reshape(-1,1)*direction

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(nr_dm[:,0], nr_dm[:,1], label='JANR')
ax.legend()
ax.set_xlabel('$O_0$')
ax.set_ylabel('$O_1$')


The numerical range of these two operators is a square with 4 vertices. We can figure out the vertices $(1,1)$ corresponds to one Bell state

$$ |\psi\rangle=\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) $$

for that

$$ \langle \psi | O_0 | \psi\rangle= \langle \psi | O_1 | \psi\rangle = 1$$

**Task**: can you identify the other three vertices?

**Task**: what about the numerical range of the operators $O_0=X\otimes X,O_1=Z\otimes I+I\otimes Z $

Then we may consider two random Hermitian operators $O_0,O_1$.

In [ ]:
op0 = numqi.random.rand_hermitian_matrix(4)
op1 = numqi.random.rand_hermitian_matrix(4)

theta_list = np.linspace(0, 2*np.pi, 400)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

beta_dm = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
nr_dm = beta_dm.reshape(-1,1)*direction

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(nr_dm[:,0], nr_dm[:,1], label='JANR')
ax.legend()
ax.set_xlabel('$O_0$')
ax.set_ylabel('$O_1$')


Since they are generated randomly, different people may get different numerical ranges. But we can still find some common features. For example, the numerical range of $O_0,O_1$ is always a convex set.

At the end of this part, to demonstrate its importance, let's quickly reproduce part of results in paper "Separability-entanglement classifier via machine learning" [doi-link](https://doi.org/10.1103/PhysRevA.98.012315). In Fig 4 of that paper, the various numerical range of two operators are shown.

$$ O_0=(\sigma_z+\sigma_0)\otimes\sigma_z/(2\sqrt{2})  $$

$$ O_1=(\sigma_y\otimes \sigma_x - \sigma_x\otimes\sigma_y)/2 $$

In the following, you will see JNAR (the boundary of the density matrix), PPT numerical range, and k-symmetrical extension numerical range. Don't get painic if you don't know what they are, we will explain them later in entanglement detection section.

In [ ]:
op0 = np.kron(np.array([[1,0],[0,0]]), numqi.gate.Z)/np.sqrt(2)
op1 = (np.kron(numqi.gate.Y, numqi.gate.X) - np.kron(numqi.gate.X, numqi.gate.Y)) / 2
theta_list = np.linspace(0, 2*np.pi, 101)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

beta_dm = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
nr_dm = beta_dm.reshape(-1,1)*direction

beta_ppt = numqi.entangle.get_ppt_numerical_range([op0, op1], direction, dim=(2,2))
nr_ppt = beta_ppt.reshape(-1,1)*direction

# about 30 seconds (on my laptop, with MOSEK)
# kext=12 would fail with SCS solver
kext_list = [2,4,6,8,10] #12
beta_kext_list = []
for kext in kext_list:
    beta_kext_list.append(numqi.entangle.get_ABk_extension_numerical_range([op0, op1], direction, dim=(2,2), kext=kext))
beta_kext_list = np.stack(beta_kext_list, axis=0)
nr_kext_list = beta_kext_list.reshape(len(kext_list),-1,1)*direction

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(nr_dm[:,0], nr_dm[:,1], label='JANR')
ax.plot(nr_ppt[:,0], nr_ppt[:,1], label='PPT')
for kext,nr_kext_i in zip(kext_list, nr_kext_list):
    ax.plot(nr_kext_i[:,0], nr_kext_i[:,1], label=f'{kext}-ext')
ax.legend()
ax.set_xlabel('$O_0$')
ax.set_ylabel('$O_1$')


One should recgnize that the orgin $(0,0)$ corresponds to the maximally mixed state $\rho_0=I/d$ for that $\mathrm{Tr}[\rho_0O_0]=\mathrm{Tr}[\rho_0O_1]=0$. The states on the boundary of the JANR are those low-rank density matrix, i.e., pure states. In the picture, the points $(\pm\frac{1}{\sqrt{2}},0)$ correspond to the pure state $|00\rangle$ and $|01\rangle$.

Task: Besides the maximally mixed state, what other states correspond to the origin or say their expectation values of these two operators are zero?

Another thing to notice is the nested structure of the k-symmetrical numerical ranges, i.e., the numerical range of $k=4$ is inside the numerical range of $k=2$, $k=6$ is inside that of $k=4$, and so on.